In [2]:
#Importação dos pacotes usados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy import signal as sg
import winsound
import time
from IPython.display import display
from ipywidgets import widgets

#Distância DTW

#conda install rpy2 #instalador do rpy2
#rpy2.robjects.r('install.packages("dtw")') #instalador do pacote dtw do R
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.functions import SignatureTranslatedFunction
rpy2.robjects.numpy2ri.activate()
R = rpy2.robjects.r
DTW = importr('dtw')

####Minha função RLE e inverso RLE numa só

def rle(value,rpt):
    from pandas import DataFrame
    n_value = len(value)
    prev = 0
    pos = 0
    lst_num = [0]
    lst_c = [0]
    for a in range(n_value):
        if value[a] == prev:
            lst_c[pos] += 1 #caso o valor seja o mesmo do anterior, acrescente mais 1 no elemento pos na lista
        else:
            prev = value[a] #muda o valor de referência
            pos += 1 #muda para o próximo valor
            lst_num.append(prev)
            lst_c.append(1)
    df = {'val': lst_num, 'rep': lst_c}
    df = pd.DataFrame(data=df)
    df.loc[df['rep'] > rpt, 'rep'] = 0
#Agora fazendo o decode
    value_2 = []
    for a in range(df.shape[0]):
        if df.iloc[a][1] > 0:
            b = df.iloc[a][1]
            for c in range(int(b)):
                value_2.append(df.iloc[a][0])
    return(np.transpose(value_2))

####AVB_register: Função de cadastro de nova voz

def AVB_register(audio, ID):
    T_frame = 1.5 #tempo em segundos de quebra
    threshold = 0.13 #Valor de identificação de nulos
    fv = [0.00625*2,0.425*2] # faixa de frequência da voz normalizado (dividido por TA)
    contador = 0
    
    #Preciso definir o tamanho do reshape
    n_colunas = int(TA*T_frame)
    zeros_add = len(audio)%n_colunas
    zeros_add = int(n_colunas - zeros_add)
    zeros_add = np.zeros(zeros_add)
    audio = np.concatenate((audio,zeros_add), axis = None)
    n_linhas = int(len(audio)/n_colunas)
    frame = audio.reshape((n_linhas, n_colunas))
    
    for a in range(n_linhas):
        audio_frame = frame[a,]
        
        #Filtro Butterworth
        
        f1, f2 = sg.butter(8, fv, btype = 'bandpass')
        audio_BW = sg.lfilter(f1, f2, audio_frame)
        
        #Filtro Offset
        
        audio_OFF = audio_BW - np.mean(audio_BW)
        
        #Normalização e removação de threshold
        
        audio_NM = audio_OFF/np.amax(audio_OFF)
        audio_NM[np.absolute(audio_NM) < threshold] = 0
        
        #RLE
        
        audio_RLE = rle(audio_NM,100)
         
        #MFCC
        
        if len(audio_RLE) > (TA/2):
            from python_speech_features import mfcc #não sei o porquê, mas funciona só se coloca aqui
            mfcc = mfcc(audio_RLE,TA)
            (n_linhas, n_colunas) = mfcc.shape
            mfcc = mfcc.reshape((1,n_linhas*n_colunas))
            
            #Adicionando o ID e apendando no MFCC geral
            
            id_mfcc = np.append(ID,mfcc)
            AVB_CDB.append(id_mfcc)
            
            contador = 1
    if contador == 1:
        print("Usuário cadastrado com sucesso")
    else:
        print("Usuário não cadastrado. Fale novamente")
            
####AVB Finder - Encontra voz numa lista cadastrada

def AVB_finder(audio, ID, AVB_CDB, precisao):
    t = time.time()
    T_frame = 0.5 #tempo em segundos de quebra
    threshold = 0.13 #Valor de identificação de nulos
    fv = [0.00625*2,0.425*2] # faixa de frequência da voz normalizado (dividido por TA)
    resposta = []
    AVB_CDB_ID = list()
    n_linhas = len(AVB_CDB)
    for a in range(n_linhas):
        if AVB_CDB[a][0] == ID:
            AVB_CDB_ID.append(AVB_CDB[a][1:])
    n_linhas_ID = len(AVB_CDB_ID)
    if n_linhas_ID == 0:
        return("Usuário não cadastrado")           
    #Preciso definir o tamanho do reshape
    n_colunas = int(TA*T_frame)
    zeros_add = len(audio)%n_colunas
    zeros_add = int(n_colunas - zeros_add)
    zeros_add = np.zeros(zeros_add)
    audio = np.concatenate((audio,zeros_add), axis = None)
    n_linhas = int(len(audio)/n_colunas)
    frame = audio.reshape((n_linhas, n_colunas))
    
    for a in range(n_linhas):
        audio_frame = frame[a,]
        
        #Filtro Butterworth
        
        f1, f2 = sg.butter(8, fv, btype = 'bandpass')
        audio_BW = sg.lfilter(f1, f2, audio_frame)
        
        #Filtro Offset
        
        audio_OFF = audio_BW - np.mean(audio_BW)
        
        #Normalização e removação de threshold
        
        max_audio = np.amax(audio_OFF)
        print("Max_audio: ",max_audio)
        if max_audio == 0.0:
            continue
        
        audio_NM = audio_OFF/max_audio #Aqui tá dando warning 148
        audio_NM[np.absolute(audio_NM) < threshold] = 0
        
        #RLE
        
        audio_RLE = rle(audio_NM,100)
        print("Tamanho_audio:", len(audio_RLE))
        
        #MFCC
        
        if len(audio_RLE) >= (TA*0.4):
            from python_speech_features import mfcc #não sei o porquê, mas funciona só se coloca aqui
            mfcc = mfcc(audio_RLE,TA)
            (n_linhas, n_colunas) = mfcc.shape
            mfcc = mfcc.reshape((1,n_linhas*n_colunas))
            
            for b in range(n_linhas_ID):
                mfcc_ID = AVB_CDB_ID[b].astype('float64')
                res_mfcc1 = rpy2.robjects.FloatVector(mfcc_ID)
                res_mfcc2 = rpy2.robjects.FloatVector(mfcc[0])
                rpy2.robjects.r('''
                    dtw_dist <- function(MFCC1, MFCC2) {
                    return(dtw(MFCC1,MFCC2)$normalizedDistance)
                    }
                    ''')
                r_f = rpy2.robjects.globalenv['dtw_dist']
                dist = r_f(res_mfcc1,res_mfcc2)
                print("Distancia:", dist[0])
                if dist[0] < precisao:
                     resposta = np.concatenate((resposta,1), axis = None)
                else:
                    resposta = np.concatenate((resposta,0), axis = None)
        else:
            resposta = np.concatenate((resposta,0), axis = None)
            print("Sem voz suficiente")
    if np.mean(resposta) > 0.5:
        return("Usuário identificado")
    else:
        return("Usuário não identificado", np.mean(resposta))
    #print("Tempo de execução: ",time.time() - t)






In [ ]:
from tkinter import *

AVB_CDB = list() #limpa lista
T_gravacao = 4 #tempo em segundos da gravação
TA = 8000 #frequência de amostragem

class Application:
    def __init__(self, master=None):
        
        self.fontePadrao = ("Arial", "12") #define uma fonte padrão
        
        self.C1 = Frame(master)
        self.C1["pady"] = 20
        self.C1.pack()
        
          
        self.C2 = Frame(master)
        self.C2["padx"] = 20
        self.C2.pack()
  
        self.C3 = Frame(master)
        self.C3["pady"] = 10
        self.C3.pack()
  
        self.C4 = Frame(master)
        self.C4["pady"] = 10
        self.C4["padx"] = 0
        self.C4.pack()
        
        self.C5 = Frame(master)
        self.C5["pady"] = 10
        self.C5["padx"] = 0
        self.C5.pack()
                
        #Título  
        self.titulo = Label(self.C1, text="!!!Bem vindo ao sistema Archangel!!!")
        self.titulo["font"] = ("Arial", "15", "bold")
        self.titulo.pack()
        
        #Nome
  
        self.menu = Label(self.C2,text="Nome", font=self.fontePadrao)
        self.menu.pack(side = LEFT)

        #Botões
  
        self.register = Button(self.C3)
        self.register["text"] = "Cadastrar nova voz"
        self.register["width"] = 17
        self.register["font"] = self.fontePadrao
        self.register["command"] = self.AVB_register
        self.register.pack(side=LEFT)
        self.finder = Button(self.C3)
        self.finder["text"] = "Encontrar minha voz"
        self.finder["font"] = self.fontePadrao
        self.finder["width"] = 17
        self.finder["command"] = self.AVB_finder
        self.finder.pack(side = LEFT)
          
       #Caixa de texto
        
        self.txtbox = Entry(self.C2)
        self.txtbox["width"] = 30
        self.txtbox["font"] = self.fontePadrao
        self.txtbox.pack(side = LEFT)      
    
        #Respostas
        
        self.beep = Label(self.C4,text="Ao som do beep diga seu nome", font=self.fontePadrao)
        self.beep.pack(side = LEFT)  
        self.answer = Label(self.C5,text="", font=self.fontePadrao)
        self.answer.pack()
    
    #Método verificar senha
    def AVB_register(self):
        usuario = self.txtbox.get()
        winsound.Beep(1800, 700)
        som = sd.rec(int(T_gravacao * TA), samplerate=TA, channels=1)
        sd.wait()
        AVB_register(som, usuario)
        self.answer["text"] = "Voz cadastrada"
    def AVB_finder(self):
        usuario = self.txtbox.get()
        winsound.Beep(1800, 700)
        som1 = sd.rec(int(T_gravacao * TA), samplerate=TA, channels=1)
        sd.wait()
        resposta = AVB_finder(som1, usuario, AVB_CDB, 6)
        self.answer["text"] = resposta
 
root = Tk()
Application(root)
root.title("archangel")
root.mainloop()

Usuário cadastrado com sucesso
Max_audio:  1.07586963397301
Tamanho_audio: 1742
Sem voz suficiente
Max_audio:  0.8770412034067002
Tamanho_audio: 2959
Sem voz suficiente
Max_audio:  1.119043604547448
Tamanho_audio: 3224
Distancia: 6.088555245246963
Distancia: 6.682363715468775
Distancia: 5.252483022608195
Max_audio:  1.1981757989954895
Tamanho_audio: 3806
Distancia: 5.621346448491316
Distancia: 6.480090141854218
Distancia: 5.191152227631506
Max_audio:  0.7407618052022629
Tamanho_audio: 1224
Sem voz suficiente
Max_audio:  1.20019826914907
Tamanho_audio: 918
Sem voz suficiente
Max_audio:  0.9157921641205397
Tamanho_audio: 3395
Distancia: 6.01703119820124
Distancia: 6.696243553790899
Distancia: 4.814056047621235
Max_audio:  0.945534362989456
Tamanho_audio: 4000
Distancia: 5.546982576511496
Distancia: 6.337621102858332
Distancia: 4.584113864828633
Max_audio:  0.0
